In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import boxcox
# import catboost
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
# reading the file

properties = pd.read_csv('rentals.csv')
properties.head()

In [ ]:
# lr = LinearRegression()
# lr.fit(x_train, y_train)
# print('R2 Squared score: ',lr.score(x_test,y_test))
# y_pred = lr.predict(x_test)
# print('Mean Squared Error: ', MSE(y_test,y_pred))
# print(lr.intercept_)

def linear_model(a,b,c,d):
    lr = LinearRegression()
    lr.fit(a,b)
    y_pred = lr.predict(c)
    Mean_error = MSE(d, y_pred)
    score = lr.score(c,d)
    RMSE = Mean_error * (1/2)
    coef = lr.coef_
    intercept = lr.intercept_
    g = f'MSE: {Mean_error}'
    h = f'R2 score: {score}'
    e = f'Coefficients: {coef}'
    f = f'Intercept: {intercept}'
    return e, f, g, h

linear_model(x_train,y_train,x_test,y_test)
    

In [ ]:
for i in properties.columns:
    if properties[f'{i}'].dtype != 'category':
            sns.boxplot(x=i, data=properties)
            plt.show()

In [ ]:
night = properties[properties['minimum_nights'] >150].index

In [ ]:
pric = properties[properties['price'] >500].index

In [ ]:
bed = properties[properties['bedrooms'] >3].index

In [ ]:
bath = properties[properties['bathrooms'] >3].index

In [ ]:
# prop = properties[properties['bathrooms'] > properties['bedrooms']].index

In [ ]:
prin = set(night).union(set(pric), set(bed), set(bath))
print(len(prin))

In [ ]:
for ing in properties.columns:
    if properties[f'{ing}'].dtype != 'category':
            sns.boxplot(x=ing, data=properties_filter)
            plt.show()

In [ ]:
properties_filter.corr()

In [ ]:
properties.corr()

In [ ]:
properties_filter.skew()

In [ ]:
for dis in properties.columns:
    if properties[f'{dis}'].dtype != 'category':
        # if dis != 'minimum_nights':
            sns.displot(properties_filter[f'{dis}'])
            plt.show()

In [ ]:
properties_filter[(properties_filter.bedrooms == 0) & (properties_filter.bathrooms==0)].head(60)

In [ ]:
len(properties_filter[properties_filter.bathrooms == 0])

In [ ]:
# 7022
properties_filter = properties_filter.drop(7022, axis=0)

In [ ]:
log_target = np.log1p(properties_filter["price"])
log_target.skew()

In [ ]:
# Removing nil prices value
properties_filter = properties_filter.drop(properties_filter[properties_filter.price <=0].index, axis=0)


In [ ]:
for trans in properties.columns:
    if properties[f'{trans}'].dtype != 'category':
        if trans == 'price' or trans == 'minimum_nights' or trans == 'latitude':
            properties_filter[f'{trans}'] = np.log1p(properties_filter[f'{trans}'])
            print(properties_filter[f'{trans}'].skew())
            sns.displot(properties_filter[f'{trans}'])
            plt.show()

In [ ]:
for trans2 in properties.columns:
    if properties[f'{trans2}'].dtype != 'category':
        if trans2 == 'bedrooms' or trans2 == 'bathrooms':
            properties_filter[f'{trans2}'] = properties_filter[f'{trans2}']**(1/2)
            print(properties_filter[f'{trans2}'].skew())
            sns.displot(properties_filter[f'{trans2}'])
            plt.show()

In [ ]:
properties_filter['longitude'] = 1/properties_filter['longitude']
sns.displot(properties_filter['longitude'])

In [ ]:
properties_filter.skew()

In [ ]:
properties_filter.corr()

In [ ]:
count_prop = properties_filter['property_type'].value_counts()

In [ ]:
count_prop[count_prop < 25]

In [ ]:
properties_filter[properties_filter['property_type'] == 'Villa']

In [ ]:
prope = properties_filter['property_type']


mask = prope.isin(count_prop[count_prop < 25].index)


prope[mask] = 'Other'
# properties_filter['property_type'] = prope.mask


In [ ]:
prope2 = prope.value_counts() 

In [ ]:
prope2[prope2 < 25].index

In [ ]:
prope3 =prope.copy()
prope3

In [ ]:
# prope4 = pd.CategoricalIndex.remove_unused_categories(properties_filter)
# prope4

In [ ]:
pd.DataFrame(prope).columns

In [ ]:
prope.value_counts()

In [ ]:
properties_filter['property_type'].unique()

In [ ]:
properties_filter[mask]

In [ ]:
# Encoding categorical values
properties_filter = pd.get_dummies(properties_filter, columns = ['property_type', 'room_type'], drop_first=True)

In [ ]:
len(properties_filter.columns)

In [ ]:
# choosing features and target variables

filter_x = properties_filter.drop('price', axis=1).values
filter_y = properties_filter['price'].values

filter_x_train, filter_x_test, filter_y_train, filter_y_test = train_test_split(filter_x, filter_y, test_size=0.2, random_state=42)
filter_x_train


In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler


transformer = pd.DataFrame(StandardScaler().fit_transform(properties_filter), columns=properties_filter.columns)

# choosing features and target variables

transformer_filter_x = transformer.drop('price', axis=1).values
transformer_filter_y = transformer['price'].values

transformer_filter_x_train, transformer_filter_x_test, transformer_filter_y_train, transformer_filter_y_test = train_test_split(transformer_filter_x, transformer_filter_y, test_size=0.2, random_state=42)


# transformer_y_train = RobustScaler().fit_transform(filter_y)
# transformer_x_test = RobustScaler().fit_transform(filter_x_test)
# transformer_y_test = RobustScaler().fit_transform(filter_y_test)
transformer_filter_x_train

In [ ]:
linear_model(transformer_filter_x_train, transformer_filter_y_train, transformer_filter_x_test, transformer_filter_y_test)

In [ ]:
linear_model(filter_x_train, filter_y_train, filter_x_test, filter_y_test)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

alp = np.arange(0,0.01,0.000001)
ride = []

for alph in alp:
    model = Ridge(alpha=alph)
    model.fit(filter_x_train, filter_y_train)
    model_pred = model.predict(filter_x_test)
    ride.append(MSE(filter_y_test,model_pred))


In [ ]:
ridges = dict(zip(ride,alp))
ridges[min(ridges.keys())]

In [ ]:
# Before transform
modelr = Ridge(alpha=1e-06)
modelr.fit(filter_x_train, filter_y_train)
model_pred = modelr.predict(filter_x_test)
mse = MSE(filter_y_test,model_pred)
score = modelr.score(filter_x_test, filter_y_test)
score2 = modelr.score(filter_x_train, filter_y_train)
print(f'MSE: {mse} \nR2_score test: {score}\nR2_score train: {score2}')


In [ ]:
# After Transform
modelr2 = Ridge(alpha=1e-06)
modelr2.fit(transformer_filter_x_train, transformer_filter_y_train)
model_pred2 = modelr2.predict(transformer_filter_x_test)
mse = MSE(transformer_filter_y_test, model_pred2)
score = modelr2.score(transformer_filter_x_test, transformer_filter_y_test)
score2 = modelr2.score(transformer_filter_x_train, transformer_filter_y_train)
print(f'MSE: {mse} \nR2_score test: {score}\nR2_score train: {score2}')


In [ ]:
from sklearn.ensemble import RandomForestRegressor

fori = []

xw = np.arange(1,50,1)

for forest in xw:
    rfr = RandomForestRegressor(n_estimators=481, max_depth=forest)
    rfr.fit(filter_x_train, filter_y_train)
    filter_pred = rfr.predict(filter_x_test)
    fori.append(MSE(filter_y_test, filter_pred))


In [ ]:
plt.figure(figsize=(17,12))
plt.plot(xw, fori)
plt.show()

In [ ]:
rfrd = dict(zip(fori,xw))
rfrd[min(rfrd.keys())]


In [ ]:
rfr = RandomForestRegressor(n_estimators=481, max_depth=25)
rfr.fit(filter_x_train, filter_y_train)
filter_pred = rfr.predict(filter_x_test)
rf_mse = MSE(filter_y_test, filter_pred)
rf = rfr.score(filter_x_test, filter_y_test)
rf2 = rfr.score(filter_x_train, filter_y_train)
print(f'MSE: {rf_mse} \nR2_score test: {rf}\nR2_score train: {rf2}')


In [ ]:
rfr = RandomForestRegressor(n_estimators=481, max_depth=25)
rfr.fit(transformer_filter_x_train, transformer_filter_y_train)
filter_pred = rfr.predict(transformer_filter_x_test)
rf_mse = MSE(transformer_filter_y_test, filter_pred)
rf = rfr.score(transformer_filter_x_test, transformer_filter_y_test)
rf2 = rfr.score(transformer_filter_x_train, transformer_filter_y_train)
print(f'MSE: {rf_mse} \nR2_score test: {rf}\nR2_score train: {rf2}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

fori2 = []

xw = np.arange(1,50,1)

for grad in xw:
    gbr = GradientBoostingRegressor(n_estimators=300, max_depth=grad)
    gbr.fit(filter_x_train, filter_y_train)
    filter_pred = rfr.predict(filter_x_test)
    fori2.append(MSE(filter_y_test, filter_pred))



In [ ]:
plt.figure(figsize=(15,9))
plt.plot(xw,fori2)
plt.show()

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=300)
gbr.fit(filter_x_train, filter_y_train)
filter_pred = rfr.predict(filter_x_test)
MSE(filter_y_test, filter_pred)

In [ ]:
gbr.score(filter_x_test, filter_y_test)

In [ ]:
gbr.score(filter_x_train, filter_y_train)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(transformer_filter_x_train, transformer_filter_y_train)
print("KNN Score: ", knr.score(transformer_filter_x_train, transformer_filter_y_train))
print("KNN Test Score: ", knr.score(transformer_filter_x_test, transformer_filter_y_test))

In [ ]:
nib = np.arange(1,100,1)
neigh = []

for neighb in nib:
    knr = KNeighborsRegressor(n_neighbors=neighb)
    knr.fit(transformer_filter_x_train, transformer_filter_y_train)
    knr_pred = knr.predict(transformer_filter_x_test)
    neigh.append(MSE(transformer_filter_y_test, knr_pred))

In [ ]:
plt.figure(figsize=(15,12))
plt.plot(nib,neigh)
plt.show()

In [ ]:
krn = dict(zip(neigh,nib))
krn[min(krn.keys())]


In [ ]:
# # Instantiate a Sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=[33], activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)

])

# # Add a Dense layer with 50 neurons and an input of 1 neuron
# model.add(Dense(50, input_shape=(35,), activation='relu'))

# # Add two Dense layers with 50 neurons and relu activation
# model.add(Dense(50,activation='relu'))
# model.add(Dense(50,activation='relu'))


# # End your model with a Dense layer and no activation
# model.add(Dense(1))

In [ ]:
filter_x_validate, filter_x_test, filter_y_validate, filter_y_test = train_test_split(filter_x_test, filter_y_test, test_size=0.5, random_state=42)

In [ ]:
# # Compile your model
model.compile(optimizer = 'adam', loss = 'mse')

print("Training started..., this can take a while:")

# # Fit your model on your data epochs
model.fit(filter_x_train, filter_y_train, epochs = 5,
          validation_data=(filter_x_validate, filter_y_validate))

# # Evaluate your model 
print("Final loss value:",model.evaluate(filter_x_test, filter_y_test))

In [ ]:
from xgboost import XGBRegressor
# xg = XGBRegressor(objective='reg:linear')
# xg.fit(filter_x_train, filter_y_train)
# preds = xg.predict(filter_x_test)

estimate = np.arange(0,20,1)
scores2 = []

for est in estimate:
    xg = XGBRegressor(learning_rate= 0.08700000000000034, n_estimators= 138, subsample= 0.8226999999999864)
    xg.fit(filter_x_train, filter_y_train)
    preds = xg.predict(filter_x_test)
    scores2.append(MSE(filter_y_test, preds))


In [ ]:
# Ridge
plt.figure(figsize=(15,8))
plt.plot(estimate,scores2)
plt.show()

In [ ]:
# estimators
plt.figure(figsize=(15,8))
plt.plot(estimate,scores2)
plt.show()

In [ ]:
# Learning rate
plt.figure(figsize=(15,8))
plt.plot(estimate,scores2)
plt.show()

In [ ]:
# subsample
plt.figure(figsize=(15,8))
plt.plot(estimate,scores2)
plt.show()

In [ ]:
mint = dict(zip(scores2,estimate))
mint[min(mint.keys())]


In [ ]:
# from xgboost import XGBRegressor

xg = XGBRegressor(learning_rate= 0.08700000000000034, n_estimators= 138, subsample= 0.8226999999999864)
xg.fit(filter_x_train, filter_y_train)
preds = xg.predict(filter_x_test)

xg_mse = MSE(filter_y_test, preds)
xg_sc = xg.score(filter_x_test, filter_y_test)*100
xg_sc2 = xg.score(filter_x_train, filter_y_train)*100
print(f'MSE: {xg_mse} \nR2_score test: {xg_sc}\nR2_score train: {xg_sc2}')

In [ ]:
# After Sacaling

xg = XGBRegressor(learning_rate= 0.08700000000000034, n_estimators= 168, subsample= 0.81)
xg.fit(transformer_filter_x_train, transformer_filter_y_train)
preds = xg.predict(transformer_filter_x_test)

xg_mse = MSE(transformer_filter_y_test, preds)
xg_sc = xg.score(transformer_filter_x_test, transformer_filter_y_test)*100
xg_sc2 = xg.score(transformer_filter_x_train, transformer_filter_y_train)*100
print(f'MSE: {xg_mse} \nR2_score test: {xg_sc}\nR2_score train: {xg_sc2}')

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'learning_rate': [0.092, 0.],
    'n_estimators': [120,130,140,150],
    'subsample': [0.89,0.9,0.91]
}


xg = XGBRegressor()
grids = GridSearchCV(estimator=xg, param_grid=parameters,
                    scoring='neg_mean_squared_error',
                    cv=5, verbose=1)
grids.fit(filter_x_train, filter_y_train)
# preds = xg.predict(filter_x_test)
grids.best_params_

In [ ]:
print(grids.best_params_)

In [ ]:
estimate = np.arange(100,200,1)
scores2 = []

for est in estimate:
    xg = XGBRegressor(learning_rate= 0.08700000000000034, n_estimators= est, subsample= 0.8226999999999864)
    xg.fit(transformer_filter_x_train, transformer_filter_y_train)
    preds = xg.predict(transformer_filter_x_test)
    scores2.append(MSE(transformer_filter_y_test, preds))

In [ ]:
# Ridge
plt.figure(figsize=(15,8))
plt.plot(estimate,scores2)
plt.show()

In [ ]:
mint = dict(zip(scores2,estimate))
mint[min(mint.keys())]
